In [152]:
!pip install transformers

In [153]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
#from tensorflow.keras import normalize
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
#import plotly.express as px
import tensorflow as tf
import pandas as pd
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
import string
from nltk.corpus import stopwords
from sklearn.metrics import f1_score
import numpy as np

In [154]:
train_df = pd.read_csv('train.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [155]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [156]:
test_df = pd.read_csv('test.csv')
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [157]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


In [158]:
#train = pd.read_csv('clean_train.csv')
#test = pd.read_csv('test.csv')
train_df['source'] = 'train'
test_df['source'] = 'test'
merged = pd.concat([train_df, test_df])

In [171]:
def getTrainSet(df):
    return df.loc[df['source'] == 'train']

def getTestSet(df):
    return df.loc[df['source'] == 'test']

In [160]:
#funciones auxiliares

def getHashtags(words):
    return ' '.join([item.lstrip('#') for item in words if item.startswith('#') and len(item) > 1])

def getMentions(words):
    return ' '.join([item.lstrip('@') for item in words if item.startswith('@') and len(item) > 1])

def getURLs(words):
    return ' '.join([item for item in words if item.startswith('http')])

def clean(text):
    
    exclude = set(string.punctuation)
    clean_text = ''.join(ch for ch in text if ch not in exclude)
    
    exclude = set(stopwords.words('english'))
    clean_text_list = clean_text.split(' ')
    clean_text = ' '.join(ch for ch in clean_text_list if ch not in exclude)

    return clean_text

#test_df['clean_text'] = test_df['text'].apply(clean)
#train_df['clean_text'] = train_df['text'].apply(clean)
#train_df.head()

In [161]:
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet


#Si tira error porque no lo reconoce ejecutar el siguiente codigo
#import nltk
#nltk.download('wordnet')

#Si tira error por no encontrar el 'averaged_perceptron_tagger' ejecutar el siguiente codigo
#nltk.download('averaged_perceptron_tagger')

def getWordNetPOSTag(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def lemmatizer(text):
    lemmatizer = WordNetLemmatizer()
    lemmaWords = []
    for w in text.split(" "):
        #lemmaWords.append(lemmatizer.lemmatize(w, getWordNetPOSTag(w)))
        lemmaWords.append(lemmatizer.lemmatize(w, pos='a')) 
        
    return " ".join(lemmaWords)

def porterStemmer(text):
    porter = PorterStemmer()
    stemmedWords = []
    for w in text.split(" "):
        stemmedWords.append(porter.stem(w)) 
        
    return " ".join(stemmedWords)

def snowballStemmer(text):
    porter = SnowballStemmer("english")
    stemmedWords = []
    for w in text.split(" "):
        stemmedWords.append(porter.stem(w)) 
        
    return " ".join(stemmedWords)

In [162]:
#FEATURES
stop = stopwords.words('english')

def crear_features(df):
    
    #texto
    df['clean_text'] = df['text'].apply(clean)
    df['lemma_text'] = df['clean_text'].apply(lemmatizer)
    df['porter_stemmed_text'] = df['clean_text'].apply(porterStemmer)
    df['snowball_stemmed_text'] = df['clean_text'].apply(snowballStemmer)
    
    #categóricas
    df['words'] = df['text'].apply(lambda x: x.split(' '))
    df['hashtags'] = df['words'].apply(getHashtags) #Obtengo los hashtags: "ht1 ht2 ht3 ..."
    df['mentions'] = df['words'].apply(getMentions) #Obtengo las menciones: "men1 men2 men3 ..."
    df['urls'] = df['words'].apply(getURLs) #Obtengo las urls "url1 url2 url3 ..."
    df['stop_words'] = df['text'].apply(lambda x: [w for w in str(x).lower().split() if w in stop])
    
    #numéricas
    df['words_count'] = df['words'].apply(lambda x: len(x))
    df['character_count'] = df['text'].str.len()
    df['mean_word_length'] = df['text'].apply(lambda x: (sum(len(w) for w in str(x).split()) / len(str(x).split())))
    df['punctuation_count'] = df['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
    df['stop_words_count'] = df['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in stop]))
    

In [163]:
crear_features(merged)
#merged.head()

In [164]:
#Keyword mean encoding
merged['keyword'].fillna('undefined', inplace=True)
kw_group = merged.groupby('keyword')['target'].mean().reset_index()
kw_group.columns = ['keyword', 'keyword_cv_mean_enc']

#Location mean encoding
merged['location'].fillna('undefined', inplace=True)
l_group = merged.groupby('location')['target'].mean().reset_index()
l_group.columns = ['location', 'location_cv_mean_enc']

#Agrego la columna keyword_cv_mean_enc y location_cv_mean_encoding
merged = pd.merge(merged, kw_group, how='left', on='keyword')
merged = pd.merge(merged, l_group, how='left', on='location')

#merged['keyword_cv_mean_enc']
#merged['location_cv_mean_enc']
merged.head()

,id,keyword,location,text,target,source,clean_text,lemma_text,porter_stemmed_text,snowball_stemmed_text,...,mentions,urls,stop_words,words_count,character_count,mean_word_length,punctuation_count,stop_words_count,keyword_cv_mean_enc,location_cv_mean_enc
0,1,undefined,undefined,Our Deeds are the Reason of this #earthquake M...,1.0,train,Our Deeds Reason earthquake May ALLAH Forgive us,Our Deeds Reason earthquake May ALLAH Forgive us,our deed reason earthquak may allah forgiv us,our deed reason earthquak may allah forgiv us,...,,,"[our, are, the, of, this, all]",13,69,4.384615,1,6,0.688525,0.424398
1,4,undefined,undefined,Forest fire near La Ronge Sask. Canada,1.0,train,Forest fire near La Ronge Sask Canada,Forest fire near La Ronge Sask Canada,forest fire near La rong sask canada,forest fire near la rong sask canada,...,,,[],7,38,4.571429,1,0,0.688525,0.424398
2,5,undefined,undefined,All residents asked to 'shelter in place' are ...,1.0,train,All residents asked shelter place notified off...,All residents asked shelter place notified off...,all resid ask shelter place notifi offic No ev...,all resid ask shelter place notifi offic no ev...,...,,,"[all, to, in, are, being, by, no, other, or, i...",22,133,5.090909,3,11,0.688525,0.424398
3,6,undefined,undefined,"13,000 people receive #wildfires evacuation or...",1.0,train,13000 people receive wildfires evacuation orde...,13000 people receive wildfires evacuation orde...,13000 peopl receiv wildfir evacu order califor...,13000 peopl receiv wildfir evacu order califor...,...,,,[in],9,65,7.125000,2,1,0.688525,0.424398
4,7,undefined,undefined,Just got sent this photo from Ruby #Alaska as ...,1.0,train,Just got sent photo Ruby Alaska smoke wildfire...,Just got sent photo Ruby Alaska smoke wildfire...,just got sent photo rubi alaska smoke wildfir ...,just got sent photo rubi alaska smoke wildfir ...,...,,,"[just, this, from, as, from, into, a]",17,88,4.500000,2,7,0.688525,0.424398


In [165]:
def trp(l, n):
    return l[:n] + [0]*(n-len(l))


In [166]:
def tokenizacion_de_texto(texto,longitud):
    marked_text = "[CLS] " + texto + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    vector = trp(indexed_tokens,longitud)
    return vector
    #return indexed_tokens

In [167]:
#train_df = getTrainSet()
#test_df = getTestSet()

#train_df = train_df.fillna('')
#test_df = test_df.fillna('')

merged = merged.fillna('')

In [168]:
#Parametros
vocab_size = 30522
epochs = 2
maxlen = 72
n_words = 500
test_size = 0.25
padding_texto = 60

In [169]:
#tokenización
def vectorizar_df(df):

    df['texto_tokenizado'] = df['text'].apply(lambda x: tokenizacion_de_texto(x,padding_texto))
    df['lemma_text_tokenizado'] = df['lemma_text'].apply(lambda x: tokenizacion_de_texto(x,padding_texto))
    df['porter_stemmed_text_tokenizado'] = df['porter_stemmed_text'].apply(lambda x: tokenizacion_de_texto(x,padding_texto))
    df['snowball_stemmed_text_tokenizado'] = df['snowball_stemmed_text'].apply(lambda x: tokenizacion_de_texto(x,padding_texto))
    df['clean_text_tokenizado'] = df['clean_text'].apply(lambda x: tokenizacion_de_texto(x,padding_texto))
    df['keyword_tokenizado'] = df['keyword'].apply(lambda x: tokenizacion_de_texto(x,5))
    df['location_tokenizado'] = df['location'].apply(lambda x: tokenizacion_de_texto(x,7))

    df['words_count'] = df['words_count'].apply(lambda x: [x])
    df['character_count'] = df['character_count'].apply(lambda x: [x])
    df['mean_word_length'] = df['mean_word_length'].apply(lambda x: [x])
    df['punctuation_count'] = df['punctuation_count'].apply(lambda x: [x])
    df['stop_words_count'] = df['stop_words_count'].apply(lambda x: [x])
    df['keyword_cv_mean_enc'] = df['keyword_cv_mean_enc'].apply(lambda x : [x])
    df['location_cv_mean_enc'] = df['location_cv_mean_enc'].apply(lambda x : [x])
    
    df['features_vectorizados'] = (  #TEXTO
                                    df['texto_tokenizado']
                                    # df['clean_text_tokenizado']
                                    #df['lemma_text_tokenizado']
                                    #df['porter_stemmed_text_tokenizado']
                                    #df['snowball_stemmed_text_tokenizado']
        
                                    #CATEGORICAS
                                    + df['keyword_tokenizado']
                                    + df['location_tokenizado']
                                  
                                    #NUMERICAS
                                    #+ df['mean_word_length']
                                    #+ df['stop_words_count']
                                    #+ df['keyword_cv_mean_enc']
                                    #+ df['location_cv_mean_enc']
                                    )

In [179]:
#vectorizar_df(merged)

#train_df = getTrainSet(merged)
#test_df = getTestSet(merged)

train_df['target']

0       1
1       1
2       1
3       1
4       1
       ..
7608    1
7609    1
7610    1
7611    1
7612    1
Name: target, Length: 7613, dtype: object

In [173]:
#split
X_train, X_test, y_train, y_test = train_test_split(train_df['features_vectorizados'], train_df['target'], test_size=test_size,
                                                  random_state=42)
FTest = test_df['features_vectorizados']
test_ids = test_df['id'] 

In [174]:
X_train = np.array([np.array(lista) for lista in X_train])
X_a = np.array([np.array(lista) for lista in X_test])

#X_train = tf.keras.utils.normalize(X_train, axis=1)
#X_a = tf.keras.utils.normalize(X_a, axis=1)

padded_FTest = X_test = np.array([np.array(lista) for lista in FTest])
padded_train = pad_sequences(X_train, maxlen = maxlen, truncating = 'post')
padded_test = pad_sequences(X_a, maxlen = maxlen, truncating = 'post')


In [175]:
#secuencia_train = tokenizar(X_train.values)
#secuencia_test = tokenizar(X_test.values)
#secuencia_FTest = tokenizar(FTest.values)

#padded_train = pad_sequences(secuencia_train, maxlen = maxlen, truncating = 'post')
#padded_test = pad_sequences(secuencia_test, maxlen = maxlen)
#padded_FTest = pad_sequences(secuencia_FTest, maxlen = maxlen)

In [176]:
#modelo Conv1D
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 128, input_length= maxlen),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv1D(256, 3, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 72, 128)           3906816   
_________________________________________________________________
dropout_6 (Dropout)          (None, 72, 128)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 70, 256)           98560     
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 32)                8224      
_________________________________________________________________
dropout_7 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                

In [177]:
model.fit(padded_train, y_train, epochs=epochs, validation_data=(padded_test, y_test))

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [ ]:
preds_locales = model.predict_classes(padded_test)
preds_locales = pd.Series(list((x[0] for x in preds_locales)))
#preds_locales

In [ ]:
print("F1 score:", f1_score(y_test, preds_locales))

In [ ]:
preds = model.predict_classes(padded_FTest)
#preds = pd.Series(preds)
preds = pd.Series(list((x[0] for x in preds)))
preds
df_preds = pd.concat([test_ids,preds],axis=1)
df_preds.rename(columns = {0 : 'target'}, inplace=True)
df_preds.set_index('id', inplace=True)
df_preds.to_csv('BERT-Conv1D-features.csv')
df_preds.head()